<a href="https://colab.research.google.com/github/Chris-Congleton/MSc-Thesis/blob/main/Analysis/TextGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 44.8 MB/s 
     |████████████████████████████████| 596 kB 44.1 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls drive/MyDrive/CS/Thesis/

 GPT2		  LDA_Trouw.png
 LDA_all.html	 'Political Influence of Dutch Newspapers.pptx'
 LDA_all.png	  rc_testdata
 LDA_NRC.html	  sample_test.html
 LDA_NRC.png	  Testset_2021
 LDA_Trouw.html   volkskrant-nl


# Finetuning

In [4]:
import pandas as pd
import re

df_NRC = pd.read_json('drive/MyDrive/CS/Thesis/Testset_2021/NRC_2021.json')
df_Volkskrant = pd.read_json('drive/MyDrive/CS/Thesis/Testset_2021/Volkskrant_2021.json')
df_Trouw = pd.read_json('drive/MyDrive/CS/Thesis/Testset_2021/Trouw_2021.json')
df_HetParool = pd.read_json('drive/MyDrive/CS/Thesis/Testset_2021/HetParool_2021.json')

dfs = [df_NRC, df_Trouw, df_HetParool, df_Volkskrant]
df = pd.concat(dfs, ignore_index=True)

df = df[df.cleantext.str.len()>5]
df = pd.DataFrame(df['cleantext'])

df['cleantext'] = df['cleantext'].map(lambda x: re.sub(r'\&\w*;', '', x))
df['cleantext'] = df['cleantext'].map(lambda x: re.sub(r'\n', '', x))
df['cleantext'] = df['cleantext'].map(lambda x: re.sub(r'\s\s+', ' ', x))
df['cleantext'] = df['cleantext'].map(lambda x: x.strip())

df = df[df.cleantext.str.len()>5]

In [5]:
df.head()

,cleantext
0,De eerste maand van 2021 is uitstekend verlope...
1,De avondklok wordt waarschijnlijk niet verleng...
2,Door de heropening van de scholen loopt het aa...
3,"Het werd alom verwacht, en zondag aan het eind..."
5,"Het was lang wachten, maar 36 jaar nadat Jan T..."


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("GroNLP/gpt2-small-dutch")
tokenizer.pad_token = tokenizer.eos_token

from transformers import TextDataset

Downloading:   0%|          | 0.00/108 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/636k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/373k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df,test_size=0.15)
print(train.shape)
print(test.shape)

(81479, 1)
(14379, 1)


In [8]:
!pip install datasets
from datasets import Dataset

dataset_train = Dataset.from_pandas(train)
dataset_test = Dataset.from_pandas(test)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 4.8 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 1.1 MB 72.6 MB/s 
     |████████████████████████████████| 140 kB 80.6 MB/s 
     |████████████████████████████████| 212 kB 78.3 MB/s 
     |████████████████████████████████| 127 kB 74.1 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 144 kB 71.4 MB/s 
     |████████████████████████████████| 271 kB 74.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1
ERROR: pip's dependency resolver does not currently take into account all t

In [9]:
max_seq_length = 128
num_proc = 4

def tokenize_function(examples):
    # Remove empty lines
    examples["cleantext"] = [line for line in examples["cleantext"] if len(line) > 0 and not line.isspace()]
    return tokenizer(
        examples["cleantext"],
        truncation=True,
        max_length=max_seq_length,
    )

train_tokenized = dataset_train.map(
    tokenize_function,
    batched=True,
    num_proc=num_proc,
    remove_columns=["cleantext"],
)

test_tokenized = dataset_test.map(
    tokenize_function,
    batched=True,
    num_proc=num_proc,
    remove_columns=["cleantext"],
)

Parameter 'function'=<function tokenize_function at 0x7f0a7a2d7560> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/21 [00:00<?, ?ba/s]

#1:   0%|          | 0/21 [00:00<?, ?ba/s]

#2:   0%|          | 0/21 [00:00<?, ?ba/s]

#3:   0%|          | 0/21 [00:00<?, ?ba/s]

#0:   0%|          | 0/4 [00:00<?, ?ba/s]

#1:   0%|          | 0/4 [00:00<?, ?ba/s]

#2:   0%|          | 0/4 [00:00<?, ?ba/s]

#3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator =  DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [11]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("GroNLP/gpt2-small-dutch")


training_args = TrainingArguments(
    output_dir="drive/MyDrive/CS/Thesis/GPT2/gpt2-General", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/457M [00:00<?, ?B/s]

In [12]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 81479
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7641
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Step,Training Loss
500,4.176400
1000,3.921700
1500,3.875000
2000,3.843700
2500,3.821300
3000,3.739800
3500,3.731200
4000,3.725900
4500,3.708900
5000,3.704800


Saving model checkpoint to drive/MyDrive/CS/Thesis/GPT2/gpt2-General/checkpoint-800
Configuration saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-General/checkpoint-800/config.json
Model weights saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-General/checkpoint-800/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/CS/Thesis/GPT2/gpt2-General/checkpoint-1600
Configuration saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-General/checkpoint-1600/config.json
Model weights saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-General/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/CS/Thesis/GPT2/gpt2-General/checkpoint-2400
Configuration saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-General/checkpoint-2400/config.json
Model weights saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-General/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/CS/Thesis/GPT2/gpt2-General/checkpoint-3200
Configuration saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-General/checkpoint-3200/config

TrainOutput(global_step=7641, training_loss=3.765811987563494, metrics={'train_runtime': 7664.9045, 'train_samples_per_second': 31.89, 'train_steps_per_second': 0.997, 'total_flos': 1.5967360106496e+16, 'train_loss': 3.765811987563494, 'epoch': 3.0})

In [13]:
trainer.save_model()

Saving model checkpoint to drive/MyDrive/CS/Thesis/GPT2/gpt2-General
Configuration saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-General/config.json
Model weights saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-General/pytorch_model.bin


# Vanilla

In [4]:
from transformers import pipeline

Vanilla = pipeline("text-generation", model="GroNLP/gpt2-small-dutch")

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/108 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/636k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/373k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

# NRC

In [3]:
from transformers import pipeline

NRC = pipeline('text-generation',model='drive/MyDrive/CS/Thesis/GPT2/gpt2-NRC', tokenizer="GroNLP/gpt2-small-dutch")

Downloading:   0%|          | 0.00/108 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/636k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/373k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

In [4]:
NRC("Mark Rutte", max_length=100, num_return_sequences=5)

[{'generated_text': 'Mark Rutte (VVD) en Sigrid Kaag (D66) willen de coronapandemie versoepelen. Dat heeft demissionair minister van Volksgezondheid Hugo de Jonge (CDA) woensdag gezegd tijdens een persconferentie, meldt persbureau ANP. �Het is voor het eerst sinds begin oktober dat je meer mogelijkheden hebt om te voorkomen dat mensen met Covid-19 besmet worden�, aldus De Jonge. Het aantal besmettingen in Nederland daalt ten opzichte van afgelopen dagen: er werden vorig jaar'},
 {'generated_text': 'Mark Rutte is de nieuwe minister van Onderwijs en vicepremier. Dat zei hij zaterdag tijdens een ingelast persconferentie in het Catshuis, meldt persbureau ANP. �De komende tijd wordt duidelijk wie we gaan doen�, aldus Rutte over zijn eerste kabinet als minister. Het gaat om staatssecretaris Paul Blokhuis (Onderwijs, D66). Hij volgt op die van Sigrid Kaag (Basis- en Voortgezet Onderwijs, VVD), Mark Rutte (VVD) en Kajsa Ollongren ('},
 {'generated_text': 'Mark Rutte (VVD) vindt dat de coronapa

#Volkskrant

In [15]:
from transformers import pipeline

Volkskrant = pipeline('text-generation',model='drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant', tokenizer="GroNLP/gpt2-small-dutch")

In [16]:
Volkskrant("Mark Rutte", max_length=100, num_return_sequences=5)

[{'generated_text': 'Mark Rutte (VVD), de VVD-leider, zei dinsdag na het kabinetsberaad: �Het was een mooie dag voor ons. We hadden nog veel te bespreken en nu zijn we klaar om op tijd weer eens naar huis te gaan. Ik wil graag dat mensen zich met rust laten genieten van wat wij hebben doorgemaakt in deze crisis.� De coronamaatregelen die Nederland vorige maand vrijdag aankondigde versoepelden tot 1 juli 2020 gelden niet langer. Maar ook wel als er sprake is van extra maatregelen'},
 {'generated_text': 'Mark Rutte (VVD), de langstzittende premier van Nederland, is al een jaar bezig zijn eigen campagne in het buitenland te voeren. In maart 2021 maakte hij zich bekend als �missionair minister� voor Buitenlandse Handel en Ontwikkelingssamenwerking. Hij kondigde dit voorjaar aan dat meer dan 100 miljard euro beschikbaar zou staan voor landen die hun grenzen niet open zullen kunnen blijven om verspreiding van besmettelijke ziekten naar Europa toe te dwingen. De afgelopen tijd was er weinig s

# Trouw

In [27]:
from transformers import pipeline

Trouw = pipeline('text-generation',model='drive/MyDrive/CS/Thesis/GPT2/gpt2-Trouw', tokenizer="GroNLP/gpt2-small-dutch")

In [ ]:
Trouw("Mark Rutte", max_length=100, num_return_sequences=5)

[{'generated_text': 'Mark Rutte en de minister van binnenlandse zaken, Sigrid Kaag (VVD�s) hebben het afgelopen jaar al gedaan. “We weten nu dat mensen in deze situatie niet goed genoeg voelen om een veilige haven te creëren waar ze kunnen terugkeren naar hun oude huis�, zegt VVD-staatssecretaris Kajsa Ollongren maandag tijdens haar wekelijkse persconferentie op Radio 1. Ze is blij met dit besluit: “Ik vind het erg jammer dat zo snel we hier zijn gekomen.'},
 {'generated_text': "Mark Rutte was al in 2017 minister van financiën. In 2018 werd hij opgevolgd door oud-bewindslieden Mark Rutte en Hedy d'Ancona. “Dat is een grote verandering�, zei de demissionair premier na het overlijden van D66-fractieleider Sigrid Kaag afgelopen maand tijdens haar afscheidsinterview met Trouw. Maar er zijn nog maar weinigen die op dit moment echt iets mee willen doen om hun eigen toekomst te laten leiden. De Tweede Kamer debatteert over"},
 {'generated_text': 'Mark Rutte zei dat hij geen voorstander is van

# Het Parool

In [40]:
from transformers import pipeline

HetParool = pipeline('text-generation',model='drive/MyDrive/CS/Thesis/GPT2/gpt2-Trouw', tokenizer="GroNLP/gpt2-small-dutch")

In [ ]:
HetParool("Mark Rutte", max_length=100, num_return_sequences=5)

[{'generated_text': 'Mark Rutte en de minister van binnenlandse zaken, Sigrid Kaag (VVD�s) hebben het afgelopen jaar al gedaan. “We weten nu dat mensen in deze situatie niet goed genoeg voelen om een veilige haven te creëren waar ze kunnen terugkeren naar hun oude huis�, zegt VVD-staatssecretaris Kajsa Ollongren maandag tijdens haar wekelijkse persconferentie op Radio 1. Ze is blij met dit besluit: “Ik vind het erg jammer dat zo snel we hier zijn gekomen.'},
 {'generated_text': "Mark Rutte was al in 2017 minister van financiën. In 2018 werd hij opgevolgd door oud-bewindslieden Mark Rutte en Hedy d'Ancona. “Dat is een grote verandering�, zei de demissionair premier na het overlijden van D66-fractieleider Sigrid Kaag afgelopen maand tijdens haar afscheidsinterview met Trouw. Maar er zijn nog maar weinigen die op dit moment echt iets mee willen doen om hun eigen toekomst te laten leiden. De Tweede Kamer debatteert over"},
 {'generated_text': 'Mark Rutte zei dat hij geen voorstander is van

# General

In [51]:
from transformers import pipeline

General = pipeline('text-generation',model='drive/MyDrive/CS/Thesis/GPT2/gpt2-General', tokenizer="GroNLP/gpt2-small-dutch")

In [52]:
General("Mark Rutte", max_length=100, num_return_sequences=5)

[{'generated_text': 'Mark Rutte (VVD) en Sigrid Kaag (D66) hebben maandag een debat gevoerd over de coronamaatregelen die in het land worden afgekondigd. Dat schrijft demissionair minister van Binnenlandse Zaken Hugo de Jonge (CDA). �Dit zijn belangrijke onderwerpen waar we elkaar niet mee aan kunnen knopen�, zei De Jonge nadat ze samen met informateur Mariëtte Hamer spraken over versoepelingen op basis van mogelijke maatregelen voor mensen zonder gevaccineerd of ongevaccineerden.'},
 {'generated_text': 'Mark Rutte (VVD) is dinsdag in de Tweede Kamer met informateur Herman Tjeenk Willink voor een spoeddebat over het coronabeleid. Dat zei hij zondag tegen journalisten. De VVD�er was er niet mee bezig om aan te kijken of dat ook �meer dan normaal� zou zijn, aldus Wopke Hoekstra (CDA).Eerder op woensdag had D66-fractievoorzitter Sigrid Kaag gezegd: “Ik ben blij dat mensen zich nu laten vaccineren'},
 {'generated_text': 'Mark Rutte (VVD) en Sigrid Kaag (D66) hebben woensdagochtend een brie

# Standard Prompt

In [24]:
Vanilla("X houdt een toespraak", max_length=50, num_return_sequences=10)

[{'generated_text': "X houdt een toespraak.,,Ik ga naar huis, maar dan kom je weer op bezoek'', zegt hij in het NOS Radio 1-programma Netwerk.\nEen groot deel van de Nederlandse publiek heeft moeite met hem en is daarom niet tevreden over zijn optreden"},
 {'generated_text': 'X houdt een toespraak in het midden van de Tweede Kamer.\nVan der Stoel: "We moeten nu eens naar buiten gaan om te kijken of we er iets mee kunnen maken."\nMaandag wordt bekend dat minister Van Mierlo (Buitenlandse Zaken'},
 {'generated_text': "X houdt een toespraak op de dag dat hij met zijn gezin naar het vliegveld is gebracht. Hij wil niet ingaan op kritiek van president Bush, maar heeft ook geen behoefte om te speculeren of er in ieder geval weer verkiezingen zullen plaatsvinden.\n'Ik heb"},
 {'generated_text': 'X houdt een toespraak. "Ik heb geen idee wat er aan het veranderen is in de samenleving", zegt hij,\'maar ik ben zeker niet van plan om hier te wonen".\nDe gemeente Amsterdam heeft zich laten inspirere

In [53]:
General("X houdt een toespraak", max_length=50, num_return_sequences=10)

[{'generated_text': 'X houdt een toespraak voor de Tweede Kamerverkiezingen op 15 november. In zijn speech, getiteld, schrijft hij dat het “belangrijke tijd is om te begrijpen wat wij allemaal moeten denken over onze toekomst en hoe we kunnen leven met deze uitdagingen�'},
 {'generated_text': 'X houdt een toespraak over de ontwikkelingen rondom het coronavirus in zijn hoofdstad, Den Haag. Op sociale media heeft hij kritiek geuit op politici die zich �niet meer bij elkaar� laten onderzoeken of ze coronavaccinaties kunnen gebruiken om te voorkomen dat'},
 {'generated_text': 'X houdt een toespraak in het bijzijn van de Amerikaanse president, Donald Trump (President) en zijn Chinese regeringsadviseur Li Xinghua. �Ik ben zeer blij dat wij deze ontmoeting hebben bereikt�, schrijft Xi op Twitter. De'},
 {'generated_text': 'X houdt een toespraak in de aanloop naar het coronatijd. Het is �het belangrijkste moment om te weten wat er gaande is�, zo heeft premier Mark Rutte vrijdag gezegd bij aflo

In [5]:
NRC("X houdt een toespraak", max_length=50, num_return_sequences=10)

[{'generated_text': 'X houdt een toespraak over de toekomst van Nederland. �Wij zijn op zoek naar wat er voor ons ligt�, zegt informateur Mariëtte Hamer (D66) in het eerste kabinet-Rutte III die woensdag aantreedt.'},
 {'generated_text': 'X houdt een toespraak tot de Tweede Kamer over coronavaccins in het buitenland. Dat meldt. De minister-president zegt �grote zorgen� te zijn dat mensen zich niet kunnen beschermen tegen Covid-19 of andere besmettingen,'},
 {'generated_text': 'X houdt een toespraak voor zijn partij in Den Haag. Hij wordt vergezeld door de fractievoorzitters, oud-minister van Defensie Ank Bijleveld (VVD) en Sigrid Kaag (D66). Voor het laatst heeft hij al gezegd dat �'},
 {'generated_text': 'X houdt een toespraak over het coronabeleid voor de Europese Unie. Dit jaar zijn er nog meer dan duizend lidstaten met zo�n coronabesmetting die hun inwoners in aanmerking zouden kunnen komen, zegt EU-voorzitter Ursula von der Leyen'},
 {'generated_text': 'X houdt een toespraak in he

In [17]:
Volkskrant("X houdt een toespraak", max_length=50, num_return_sequences=10)

[{'generated_text': 'X houdt een toespraak op de Dam in Amsterdam-Zuidoost. De nieuwe burgemeester Femke Halsema (VVD): �We hebben geen idee wat we moeten doen als onze stad weer wordt overspoeld door mensen van wie je niet kent.� En: �'},
 {'generated_text': 'X houdt een toespraak. �Ik vind dat het goed gaat voor iedereen�, zei hij zondag tijdens zijn verkiezingsavond in de Amsterdamse stadsdeel Centrum. De partijtop had nog geen idee wat er te doen was met deze coronapandemie'},
 {'generated_text': 'X houdt een toespraak in de Grote Kerk van Amsterdam. �De coronapandemie is voorbij en we zijn nu bezig met deze moeilijke fase�, vertelt hij op Twitter. De nieuwe coronavirus komt niet tot stand omdat er geen vaccin beschikbaar'},
 {'generated_text': 'X houdt een toespraak in het centrum van Beijing, waar de Chinese president Xi Jinping al sinds juni zijn intrek heeft genomen. �We zullen nooit vergeten wat er met hem aan de macht is�, twitterde hij afgelopen zaterdag op Twitter.'},
 {'ge

In [28]:
Trouw("X houdt een toespraak", max_length=50, num_return_sequences=10)

[{'generated_text': 'X houdt een toespraak voor het eerst sinds zijn aantreden in 2018. “Ik ben heel tevreden over wat ik heb gedaan�, schrijft hij op Twitter. Het is niet de enige keer dat premier Rutte en minister van buitenlandse zaken Sigrid K'},
 {'generated_text': 'X houdt een toespraak in het Witte Huis. “Ik ben er niet voor gekomen, maar ik moet wel naar buiten gaan�, aldus president Joe Biden (defensie). Hij zei dit terwijl hij aan de telefoon met premier Mark Rutte'},
 {'generated_text': 'X houdt een toespraak in het centrum van Istanbul, waar hij zich voor de eerste keer uitspreekt over �de strijd om democratie en vrijheid op aarde�. Een paar uur na zijn beëdiging zegt hij nog eens tegen journalisten: “Ik'},
 {'generated_text': 'X houdt een toespraak. “Het is de laatste tijd dat mensen met psychische problemen het gevoel krijgen dat ze op vakantie of in hun werk niet kunnen worden geholpen�, legt hij uit. De afgelopen twee jaar zijn meer dan 70 procent van'},
 {'generated_te

In [41]:
HetParool("X houdt een toespraak", max_length=50, num_return_sequences=10)

[{'generated_text': 'X houdt een toespraak over de �hervormingen van onze samenleving�. Hij stelt dat er meer ruimte moet worden voor reflectie op het heden, waarin we ons niet alleen moeten zien of veranderen maar ook beter en gelukkiger maken. In zijn speech'},
 {'generated_text': 'X houdt een toespraak. “In onze tijd hebben we alle vrijheid die er op dit moment is, maar het kan niet anders dan dat mensen hun verantwoordelijkheid moeten nemen voor de gevolgen daarvan�, vertelt ze in gesprek met Het Financieele'},
 {'generated_text': 'X houdt een toespraak op de Dam aan het Binnenhof. In zijn speech staat hij vast: “Het is voor mij belangrijk dat er zoveel mogelijk mensen aanwezig zijn om over te praten met ons�. Het kabinet-Rutte II, die'},
 {'generated_text': 'X houdt een toespraak over de toekomst van zijn land. “Er is weinig veranderd�, zegt premier Mark Rutte in het interview met De Nederlandsche Bank (DNB). “Je zou denken: wat hebben we er nog aan'},
 {'generated_text': 'X houdt

# Article rewriting

**Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie** misschien wel „het best uit deze economische crisis komt van de hele westerse wereld”. **Dat zei Mark Rutte (VVD) vrijdag** na afloop van de Eurotop in Brussel tegen persbureau ANP. **Het is een spannende race, zet Rutte,** „maar we zitten in ieder geval in de kopgroep”. **Tegen de achtergrond van de ernstige gevolgen van de coronacrisis** staat Nederland er volgens Rutte „heel goed” voor.

**Na de zomer kunnen landen uitbetaald worden** vanuit het coronaherstelfonds dat de Europese Unie vorig jaar heeft opgetuigd. **Het demissionair kabinet heeft volgens Rutte** nog geen herstelplan opgesteld om naar Brussel te sturen. **Dat moet de nieuwe regering** doen na de formatie. **Volgens de demissionair premier** maakt het niet uit dat het plan nog niet ingediend is. „Dat geld is er gewoon”, zei hij vrijdag. **Als de formatie te lang duurt,** zal het demissionaire kabinet zich er volgens Rutte mogelijk wel over buigen.

## Vanilla

In [6]:
Vanilla("Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie", max_length=30, num_return_sequences=5)

[{'generated_text': "Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie nog op een andere manier tegemoet. 'Het gaat om"},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie niet meer voor.\nOp een persconferentie zei hij dat'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie nog niet in actie tegen het coronavirus. "We hebben'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie nog in gesprek met het RIVM over een nieuwe aanpak.'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie nog eens uit te spreken over nieuwe maatregelen tegen verspreiding van'}]

In [7]:
Vanilla("Dat zei Rutte vrijdag", max_length=30, num_return_sequences=5)

[{'generated_text': "Dat zei Rutte vrijdag op een persconferentie in Den Haag: 'Er is niet veel nieuws aan de hand.'\nDe premier had al gezegd dat hij het"},
 {'generated_text': "Dat zei Rutte vrijdag na een debat in de Tweede Kamer. Volgens hem is het kabinet 'een grote stap voorwaarts gegaan'.\nPremier Mark Rutte heeft"},
 {'generated_text': "Dat zei Rutte vrijdag op zijn persconferentie: 'Ik denk wel dat ik er niet aan toe heb nagedacht.'\nIn het NOS-interview benadrukte de premier"},
 {'generated_text': 'Dat zei Rutte vrijdag in het NOS Radio 1 Journaal: "Ik vind dat ik er wel over heb gesproken. Ik ben nu nog niet zo zeker van'},
 {'generated_text': 'Dat zei Rutte vrijdag in een reactie aan de Tweede Kamer.\n"Ik vind het zeer betreurenswaardig dat ik zo\'n politiek gevoelige opmerking maak",'}]

In [8]:
Vanilla("Het is een spannende race, zet Rutte,", max_length=30, num_return_sequences=5)

[{'generated_text': "Het is een spannende race, zet Rutte, maar ik wil graag met de mensen in het kabinet blijven.'\nDe partij van VVD-leider Mark Rutte"},
 {'generated_text': 'Het is een spannende race, zet Rutte, maar ik hoop dat hij de juiste keuzes kan nemen om het tij te keren. Ik heb mijn twijfels over'},
 {'generated_text': "Het is een spannende race, zet Rutte, maar we zijn daar nog niet klaar voor.'\nPoirot heeft ook kritiek op het kabinetsberaad."},
 {'generated_text': "Het is een spannende race, zet Rutte, maar ik denk er niet uit dat hij het gaat lukken.'\nD66-lijsttrekker Alexander Pechtold heeft"},
 {'generated_text': "Het is een spannende race, zet Rutte, maar ik vind het lastig.'\n'Ik kan me niet voorstellen dat er in dit land iets gebeurt waar"}]

In [9]:
Vanilla("Tegen de achtergrond van de ernstige gevolgen van de coronacrisis", max_length=30, num_return_sequences=5)

[{'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn veel Nederlanders bang om het leven te redden. In sommige landen hebben mensen die niet in nood verkeren'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis is het niet alleen een kwestie van tijd. Er zijn ook nog veel andere dingen te kunnen gebeuren,'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn er tal van nieuwe maatregelen nodig. Het aantal ziekenhuisopnames moet worden teruggebracht naar twee per week,'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis is ook dat er niet veel aandacht wordt besteed aan maatregelen voor mensen met een psychische aandoening zoals ADHD'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis is er nog iets nieuws te melden. Het aantal coronapandemiegevallen stijgt met een kwart'}]

In [10]:
Vanilla("Na de zomer kunnen landen uitbetaald worden", max_length=30, num_return_sequences=5)

[{'generated_text': 'Na de zomer kunnen landen uitbetaald worden.\n"Ik denk dat we ons moeten laten zien hoe het gaat", zei Van der Steur in een brief aan'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden, maar ze hoeven er nog niet veel geld voor te betalen.\nDe Europese Centrale Bank (ECB) wil'},
 {'generated_text': "Na de zomer kunnen landen uitbetaald worden. Zo'n 1,9 miljard dollar per jaar is nog niet veel meer dan een kwart van het bruto binnenlands"},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden. Dat geldt niet alleen voor Italië en Frankrijk, maar ook voor alle EU-lidstaten die zich moeten toeleggen'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden.\nOok in Duitsland is het nog steeds lastig om er geld uit te wisselen voor projecten die aan een ander land'}]

In [11]:
Vanilla("Het demissionair kabinet heeft volgens Rutte", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het demissionair kabinet heeft volgens Rutte en zijn adviseurs de afgelopen twee jaar een overschot van 4,2 miljard euro aan defensie.\nOp dit moment'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte en Kamervoorzitter Arib een plan om de begroting in evenwicht te houden, maar dat is het enige doel.'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte de afgelopen maanden veel te druk uitgeoefend op minister Van der Steur van Veiligheid en Justitie. Het is niet ondenkbaar dat'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte een eigen verantwoordelijkheid genomen. Dat is in strijd met het regeerakkoord dat minister Van Aartsen (VVD) eerder deze week'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte het vertrouwen op de Europese veiligheid in gevaar gebracht. "Ik ben bang dat we daar niet meer zijn."\n'}]

In [12]:
Vanilla("Dat moet de nieuwe regering", max_length=30, num_return_sequences=5)

[{'generated_text': "Dat moet de nieuwe regering van premier Tsipras nog goedkeuren.'\n'Hij heeft al gezegd dat hij geen akkoord zal komen', zegt correspondent Michel Barnier"},
 {'generated_text': "Dat moet de nieuwe regering ook zijn.'\n'Ja. Maar we hebben het wel een paar maanden geleden besproken en ik ben niet in staat om te"},
 {'generated_text': "Dat moet de nieuwe regering niet doen.'\n'Maar ik geloof dat u het wilt proberen,' zei hij met een zucht. 'Het zou heel goed"},
 {'generated_text': "Dat moet de nieuwe regering wel. Het is nu een kwestie van tijd.'\n'Je hebt gelijk, het lijkt erop dat er iets aan de hand"},
 {'generated_text': 'Dat moet de nieuwe regering van premier Rutte niet zijn geweest. De afgelopen jaren is er nog veel meer over het klimaatbeleid en hoe dat kan worden verbeterd'}]

In [13]:
Vanilla("Volgens de demissionair premier", max_length=30, num_return_sequences=5)

[{'generated_text': "Volgens de demissionair premier heeft hij niet veel meer te zeggen. 'We hebben geen enkel idee hoe het in Duitsland is.'\nDe SPD-"},
 {'generated_text': 'Volgens de demissionair premier is het nog niet duidelijk of zijn coalitiepartners, VVD en PvdA, akkoord gaan.\nPvdA-leider Diederik Samsom heeft'},
 {'generated_text': "Volgens de demissionair premier is er geen sprake van een'sociale coalities' tussen Nederland en Groot-Brittannië. Premier Blair zegt dat"},
 {'generated_text': 'Volgens de demissionair premier van Frankrijk, Francois Mitterrand, zal dat niet gebeuren. "Het is een politieke crisis", zei hij gisteren in Parijs'},
 {'generated_text': "Volgens de demissionair premier Kok is het nog niet in zicht.,,Ik ben wel blij dat we nu een kabinet hebben gevormd'', zei Dijkstal gisteren"}]

In [14]:
Vanilla("Als de formatie te lang duurt,", max_length=30, num_return_sequences=5)

[{'generated_text': "Als de formatie te lang duurt, dan kunnen we er niet omheen. Dat is een mooie gedachte.'\n'Dat wil ik ook nog wel,' zei"},
 {'generated_text': "Als de formatie te lang duurt, zal er nog wel sprake van een breuk worden tussen VVD en PVV'er Van den Brink.\nDe PvdA-"},
 {'generated_text': "Als de formatie te lang duurt, kan er ook nog wel wat gebeuren.'\nVan den Berg: 'We hebben het een beetje druk gemaakt. We"},
 {'generated_text': 'Als de formatie te lang duurt, zal er een nieuw kabinet komen", zei minister Van Boxtel van Volkshuisvesting en Ruimtelijke Ordening in reactie.\n'},
 {'generated_text': "Als de formatie te lang duurt, dan zal het nog een paar maanden duren. We zijn er niet helemaal zeker van.'\nVan den Berg: '"}]

##General

In [21]:
General("Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie", max_length=30, num_return_sequences=5)

[{'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie weer in gesprek over het vaccineren. �We'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie een derde kabinet met �een duidelijke boodschap�'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie in lockdown. De minister van Economische Zaken laat weten'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie een �nieuwe orde� op het gebied van'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie nog eens met �grote zorgen� over het'}]

In [22]:
General("Dat zei Rutte vrijdag", max_length=30, num_return_sequences=5)

[{'generated_text': 'Dat zei Rutte vrijdag in een debat met informateur Mariëtte Hamer. “Ik begrijp dat er vragen rijzen over de inhoud, maar'},
 {'generated_text': 'Dat zei Rutte vrijdag tijdens een persmoment over het coronabeleid in Den Haag. �Het kabinet wil dat er meer mensen op de intensive care'},
 {'generated_text': 'Dat zei Rutte vrijdag tijdens de wekelijkse ministerraad. De VVD en D66 zouden samen met CDA-Kamerlid Pieter Omtzigt een motie in willen indienen om'},
 {'generated_text': 'Dat zei Rutte vrijdag tijdens een persconferentie. “Ik ben er zeker van dat we het goed kunnen doen�, voegde hij eraan toe.'},
 {'generated_text': 'Dat zei Rutte vrijdag tijdens de Algemene Politieke Beschouwingen. �We gaan er nu weer over praten�, aldus het demissionaire kabinet'}]

In [23]:
General("Het is een spannende race, zet Rutte,", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het is een spannende race, zet Rutte, �een uitdaging� voor de Nederlandse politiek. De VVD-leider wil dat er op dit moment'},
 {'generated_text': 'Het is een spannende race, zet Rutte, die zich in de Tweede Kamer heeft gestort op �de juiste koers�. De VVD-leider'},
 {'generated_text': 'Het is een spannende race, zet Rutte, maar toch zijn we er al heel lang aan gewend. “We hebben in de afgelopen 24 uur'},
 {'generated_text': 'Het is een spannende race, zet Rutte, maar hij heeft er wel genoeg van. “Maar de vraag is: wat hebben we gewonnen?'},
 {'generated_text': 'Het is een spannende race, zet Rutte, en dan gaat het even stil. Het lijkt erop dat de premier niet van plan is zich te laten vaccin'}]

In [24]:
General("Tegen de achtergrond van de ernstige gevolgen van de coronacrisis", max_length=30, num_return_sequences=5)

[{'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis is het niet verwonderlijk dat veel mensen met een chronische ziekte worden gevaccineerd. Dat blijkt uit onderzoek'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn er ook veel zorgen. “Het is voor ons allemaal een zware bevalling�,'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn tal van maatregelen te versoepelen, zegt demissionair minister Wopke Hoekstra (Financiën'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis, dat in veel gevallen ook voor kinderen en jonge volwassenen is. Zo�n 80 procent van'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis is nog veel te zien. In het afgelopen jaar zijn er meer dan 100 duizend nieuwe besmettingen gemeld'}]

In [25]:
General("Na de zomer kunnen landen uitbetaald worden", max_length=30, num_return_sequences=5)

[{'generated_text': 'Na de zomer kunnen landen uitbetaald worden in het coronatoegangsbewijs. Wie een negatieve testuitslag heeft, kan zich laten vaccineren als dat veilig is'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan het aantal coronabesmettingen in Nederland. Dat demissionair minister Hugo de Jonge (Volksgezondheid, CDA'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan coronavaccins. Dat meldt het Centraal Bureau voor de Statistiek (CBS) vrijdagochtend op basis van cijfers'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan mensen die hun baan hebben opgegeven. Dat heeft premier Mark Rutte (VVD) woensdag bekendgemaakt, meldt persbureau ANP.'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan migranten die uit Afghanistan hebben verdreven. Dat heeft premier Justin Trudeau dinsdag gezegd in een toespraak, melden internationale'}]

In [26]:
General("Het demissionair kabinet heeft volgens Rutte", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het demissionair kabinet heeft volgens Rutte �nog niet� besloten tot versoepelingen van het coronamaatregelen. Dat liet premier Mark Rutte'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte en D66 een �grote stap voorwaarts gezet� gemaakt in het aanpakken van de toeslagenaffaire.'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte �ernstige plannen� gemaakt om het aantal coronapatiënten in Nederland te verminderen. Dat'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte nog niet besloten wat er in het nieuwe regeerakkoord precies moet komen. Dat blijkt uit een brief naar Tweede Kamervoorzitter'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte en de Kamer ingestemd met het invoeren van een coronamaatregelenpakket. Dat maakte premier Mark Rutte (VVD'}]

In [27]:
General("Dat moet de nieuwe regering", max_length=30, num_return_sequences=5)

[{'generated_text': 'Dat moet de nieuwe regering van Myanmar, die eerder dit jaar aantrad als interim-president, nu nog eens aanstellen. Volgens het Myanmare'},
 {'generated_text': 'Dat moet de nieuwe regering-Rutte IV in een week tijd besluiten. De demissionaire minister van Buitenlandse Zaken Stef Blok (ChristenUnie)'},
 {'generated_text': 'Dat moet de nieuwe regering-Rutte III van Rutte en D66 hebben opgeleverd. Dat zei demissionair minister Kajsa Ollongren (Binnen'},
 {'generated_text': 'Dat moet de nieuwe regering wel lukken. �We hebben het afgelopen jaar zo�n 1,8 miljard euro nodig om dit land te redden'},
 {'generated_text': 'Dat moet de nieuwe regering in het bijzonder doen, zei demissionair premier Mark Rutte maandag na een debat met informateur Mariëtte Hamer.'}]

In [28]:
General("Volgens de demissionair premier", max_length=30, num_return_sequences=5)

[{'generated_text': 'Volgens de demissionair premier heeft het �heel zwaar� op zich genomen en zal er geen enkele versoepeling van coronamaatregelen worden'},
 {'generated_text': 'Volgens de demissionair premier, Sigrid Kaag (D66), is het tijd voor een nieuw kabinet. Dat heeft ze woensdagavond bekendgemaakt op Twitter'},
 {'generated_text': 'Volgens de demissionair premier is het aantal coronabesmettingen en ziekenhuisopnames in Nederland al sinds eind maart met 2,9 procent gedaald. Dat'},
 {'generated_text': 'Volgens de demissionair premier wordt er een nieuw kabinet opgericht om te zorgen dat meer mensen zich laten vaccineren. Dat heeft minister van Volksgezondheid Hugo de'},
 {'generated_text': 'Volgens de demissionair premier is het aantal coronapatiënten dat op een ICafspraak in Amsterdam opgenomen moet worden �overvol�'}]

In [29]:
General("Als de formatie te lang duurt,", max_length=30, num_return_sequences=5)

[{'generated_text': 'Als de formatie te lang duurt, is het dan ook tijd dat we een nieuw kabinet vormen. De huidige premier Mark Rutte kan nog steeds rekenen op steun'},
 {'generated_text': 'Als de formatie te lang duurt, dan is er geen sprake van een nieuw kabinet. Dat zei demissionair minister De Jonge (Volksgezondheid, CDA'},
 {'generated_text': 'Als de formatie te lang duurt, is dat voor VVD en D66 niet verwonderlijk. Beide partijen zijn bezig met “veranderend�'},
 {'generated_text': 'Als de formatie te lang duurt, komt er een einde aan het politieke leven. Er zijn twee partijen die met elkaar kunnen praten: VVD en D66'},
 {'generated_text': 'Als de formatie te lang duurt, dan komt er nog eens een nieuwe premier. De VVD-kiezers hebben al genoeg van Rutte en zijn kabinet'}]

## NRC

In [6]:
NRC("Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie", max_length=30, num_return_sequences=5)

[{'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie weer terug op het normale leven. Dat meldt persbureau ANP'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie op een nieuw hoogtepunt. Dat heeft het kabinet woensdag bekendgemaakt'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie weer tot bezinning over het coronavirus. Dat zei hij tijdens'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie nog steeds in lockdown. �Dit is een'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie een nieuw kabinet. Dat zei hij tijdens het Catshuis'}]

In [7]:
NRC("Dat zei Rutte vrijdag", max_length=30, num_return_sequences=5)

[{'generated_text': 'Dat zei Rutte vrijdag tijdens een persmoment over de coronapandemie. �Daar ben ik van overtuigd en dat is wel zo�'},
 {'generated_text': 'Dat zei Rutte vrijdag na afloop van de ministerraad  �heel erg goed� over het coronabeleid. Hij is op zoek naar een nieuwe'},
 {'generated_text': 'Dat zei Rutte vrijdag na afloop van de eerste ministerraad: �Ik ben er niet om aan te denken.� Dat was een grapje, maar'},
 {'generated_text': 'Dat zei Rutte vrijdag tegen journalisten. �Wij staan op de valreep van een nieuwe crisis�, zo schreef hij in het tv-programma'},
 {'generated_text': 'Dat zei Rutte vrijdag na afloop van de Algemene Politieke Beschouwingen (, 22/7). Hij benadrukte dat we ons �heel goed� moeten'}]

In [8]:
NRC("Het is een spannende race, zet Rutte,", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het is een spannende race, zet Rutte, Klaver en Wilders op de juiste weg naar het parlement. Het lijkt erop dat VVD-leider Mark Rutte ('},
 {'generated_text': 'Het is een spannende race, zet Rutte, Klaver en D66 op scherp in de aanval. �Ik weet zeker dat dit het belangrijkste verkiezingsdebat'},
 {'generated_text': 'Het is een spannende race, zet Rutte, D66 en het CDA de achtervolging in om te gaan regeren. �Ik wil niet zeggen dat ik'},
 {'generated_text': 'Het is een spannende race, zet Rutte, maar het kan toch niet anders. Een week na de Tweede Kamerverkiezingen wordt er opnieuw �opnieuw�'},
 {'generated_text': 'Het is een spannende race, zet Rutte, D66 en ChristenUnie in de weg. Het gaat om  �de grootste coalitie van Europa�'}]

In [9]:
NRC("Tegen de achtergrond van de ernstige gevolgen van de coronacrisis", max_length=30, num_return_sequences=5)

[{'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn in het Verenigd Koninkrijk ruim 2.800 coronagevallen gemeld. Dat heeft demissionair premier'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn veel mensen positief getest op corona.  Dat meldt het Rijksinstituut voor Volksgezondheid en Milieu (R'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn tal van positieve ontwikkelingen te zien op het gebied van coronatests. Dat blijkt vrijdag uit'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn er steeds nieuwe maatregelen te overwegen. Dat laat het ministerie van Volksgezondheid, Welzijn en Sport ('},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn er grote demonstraties in Den Haag. Dat melden  woensdag op basis van informatie die demissionair'}]

In [10]:
NRC("Na de zomer kunnen landen uitbetaald worden", max_length=30, num_return_sequences=5)

[{'generated_text': 'Na de zomer kunnen landen uitbetaald worden voor coronavaccins. Dat heeft het demissionaire kabinet woensdag  in een persconferentie, zo meldt persbureau ANP'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan coronapatiënten. Dat zegt een woordvoerder van het Internationale Patiëntenfederatie (ICF). Het ICF'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan coronapatiënten dat in een ziekenhuis verblijven, zo heeft het Internationaal Monetair Fonds (IMF).'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan coronavaccins, zo heeft het Amerikaanse ministerie van Volksgezondheid donderdag. Dat meldt persbureau Reuters. In totaal krijgen'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan coronapatiënten die in het buitenland verblijven. Dat heeft premier Mark Rutte (VVD) woensdag gezegd tijdens een'}]

In [11]:
NRC("Het demissionair kabinet heeft volgens Rutte", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het demissionair kabinet heeft volgens Rutte III de �stabiliteitsagenda� bereikt in het buitenland. Dat melden internationale persbureaus op basis'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte III nog maar twee jaar aan te sluiten, omdat de coronamaatregelen onvoldoende zijn om het aantal besmettingen'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte III ingestemd met het versoepelingen van coronamaatregelen bij evenementen en concerten. Dat meldt persbureau ANP op basis'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte een �grote kans� op de vorming van een nieuwe regering, zo laat het Rijksinstituut voor Volksgezondheid'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte �heel goede hoop� te hebben gedaan om  het coronavaccin van Pfizer en BioN'}]

In [12]:
NRC("Dat moet de nieuwe regering", max_length=30, num_return_sequences=5)

[{'generated_text': 'Dat moet de nieuwe regering in Den Haag niet meer doen. Dat laat demissionair minister van Defensie Ank Bijleveld (Defensie, VVD'},
 {'generated_text': 'Dat moet de nieuwe regering van premier Mark Rutte zijn. Dat zei hij in een interview met. �In het huidige kabinet worden we nog niet eens'},
 {'generated_text': 'Dat moet de nieuwe regering-Ruimtelijke Ordening, die donderdag in het kabinet wordt geïnstalleerd, wel eens worden goedgekeurd. Dat heeft demissionair'},
 {'generated_text': 'Dat moet de nieuwe regering van premier Boris Johnson (Rutte III) wel lukken om het klimaatakkoord van Parijs te bereiken. Dat heeft demissionair'},
 {'generated_text': 'Dat moet de nieuwe regering van president Donald Trump zijn. Dat zei demissionair minister Hugo de Jonge (Volksgezondheid, CDA) donderdag na afloop van'}]

In [13]:
NRC("Volgens de demissionair premier", max_length=30, num_return_sequences=5)

[{'generated_text': 'Volgens de demissionair premier Mark Rutte (VVD) is het �ernstigst getroffen� om te voorkomen dat burgers die in coronat'},
 {'generated_text': 'Volgens de demissionair premier Mark Rutte (VVD) is het coronabeleid niet meer nodig. In een gesprek met informateur Mariëtte H'},
 {'generated_text': 'Volgens de demissionair premier Mark Rutte (VVD) is er een tekort aan bedden van coronavaccinaties. �Dat betekent niet dat we'},
 {'generated_text': 'Volgens de demissionair premier Mark Rutte (VVD) en staatssecretaris Mona Keijzer (Medische Zorg, CDA) is er geen sprake van �'},
 {'generated_text': 'Volgens de demissionair premier Mark Rutte (VVD) en zijn collega�s in Den Haag is een nieuw kabinet mogelijk. De ministers van Financiën'}]

In [14]:
NRC("Als de formatie te lang duurt,", max_length=30, num_return_sequences=5)

[{'generated_text': 'Als de formatie te lang duurt, zal er nog steeds wel een coalitie ontstaan tussen PvdA-lijsttrekker Asscher en CDA-leider Wopke Hoekstra.'},
 {'generated_text': 'Als de formatie te lang duurt, wordt er een nieuw kabinet gevormd. Dat maakte premier Rutte  dinsdag bekend aan het begin van zijn persconferentie –'},
 {'generated_text': 'Als de formatie te lang duurt, zal een meerderheid in het Tweede Kamer zijn. Dat zei demissionair minister van Binnenlandse Zaken Kajsa Ollongren'},
 {'generated_text': 'Als de formatie te lang duurt, is er in ieder geval een nieuw kabinet nodig. Dat heeft het Centraal Bureau voor de Statistiek (CBS) donderdag laten'},
 {'generated_text': 'Als de formatie te lang duurt, zal er een einde komen aan het kabinetsbeleid. �We zijn niet klaar�, zei demissionair'}]

## Volkskrant

In [29]:
Volkskrant("Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie", max_length=30, num_return_sequences=5)

[{'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie al in actie. �Dit is een historisch moment'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie een jaar of twee terug bij het aantal besmettingen.'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie een jaar lang niet verder dan 1.500 besmettingen'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie in lockdown te vervallen. In plaats van een �'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie weer in beeld. �Er is een einde gekomen'}]

In [19]:
Volkskrant("Dat zei Rutte vrijdag", max_length=30, num_return_sequences=5)

[{'generated_text': 'Dat zei Rutte vrijdag in een verklaring na afloop van de kabinetsvergadering. �Er zijn nog veel mensen die het niet willen�, hij voegde'},
 {'generated_text': 'Dat zei Rutte vrijdag tijdens een persconferentie in het Catshuis. �We laten weten dat we dit niet langer kunnen doorgaan�, aldus de premier'},
 {'generated_text': 'Dat zei Rutte vrijdag in een toelichting aan de Algemene Politieke Beschouwingen (APB). Hij voegde daaraan toe dat er �grote zorgen�'},
 {'generated_text': 'Dat zei Rutte vrijdag tijdens de Algemene Politieke Beschouwingen (APB). �Er is iets wat we in Nederland hebben geleerd.� De'},
 {'generated_text': 'Dat zei Rutte vrijdag na een debat in het Kamerdebat met de demissionair staatssecretaris van Financiën (Financiën). �Ik ben ervan overtuigd dat'}]

In [20]:
Volkskrant("Het is een spannende race, zet Rutte,", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het is een spannende race, zet Rutte, D66-leider Sigrid Kaag en zijn secondanten op het verkeerde been. De formatie van de'},
 {'generated_text': 'Het is een spannende race, zet Rutte, D66 en ChristenUnie op hun kop. Het was voor het eerst in bijna tien jaar tijd dat de formatie'},
 {'generated_text': 'Het is een spannende race, zet Rutte, zijn vrouw en de hele natie op hun schreden terug. �Ik weet het niet�, zegt'},
 {'generated_text': 'Het is een spannende race, zet Rutte, Klaver en D66 in de verkeerde richting. Het lijkt alsof iedereen zich afvraagt waarom het niet goed komt om'},
 {'generated_text': 'Het is een spannende race, zet Rutte, Klaver en D66 in de eerste helft van het jaar hun eigen koers aan. �Op dit moment'}]

In [21]:
Volkskrant("Tegen de achtergrond van de ernstige gevolgen van de coronacrisis", max_length=30, num_return_sequences=5)

[{'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn er in Nederland meer dan 150 duizend besmettingen per dag. Het aantal nieuwe coronabesmettingen'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis is het nog steeds lastig om een nieuw kabinet tot stand te brengen. Er staan al twee partijen die'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis hebben veel mensen zich afgevraagd of het mogelijk was dat ze na corona alsnog een uitkering zouden kunnen krijgen.'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis hebben we al een tijdje niet veel tijd om over te schrijven. Maar nu gaat het ook heel anders'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis is er veel onrust in Nederland. De coronamaatregelen worden door het kabinet al aangekondigd, maar'}]

In [22]:
Volkskrant("Na de zomer kunnen landen uitbetaald worden", max_length=30, num_return_sequences=5)

[{'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan het ministerie van Financiën. Het geld komt niet direct uit, maar is bedoeld voor mensen die in aanmerking komen voor'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan burgers die hun leven in een ander land hebben geruïneerd. Nu is dat niet gebeurd, blijkt uit cijfers'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden. Dat is niet alleen mogelijk dankzij het coronabeleid, maar ook door �de� versoepelingen die'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden in euro�s. Het was al een verrassing toen het Amerikaanse ministerie van Financiën (BON) dit jaar'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden voor �s werelds meest vervuilende industrie. Dat is niet te verwachten als er geen maatregelen zijn om die uitstoot'}]

In [23]:
Volkskrant("Het demissionair kabinet heeft volgens Rutte", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het demissionair kabinet heeft volgens Rutte al een plan om de coronapandemie op te pakken. Maar als dat niet lukt, moet het'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte en D66 afgesproken dat het aantal coronabeleid-besmettingen de komende weken nog zal stijgen. De'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte en zijn collega�s afgesproken dat alle coronabeleidmaatregelen die de afgelopen twee weken gelden, zullen worden'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte al ingestemd met versoepelingen. Het CDA, de VVD en D66 willen dat er in 2050 zo�'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte de komende tijd een aantal maatregelen aangekondigd om te voorkomen dat het stikstofprobleem opnieuw kan worden opgelost. Een groot deel'}]

In [24]:
Volkskrant("Dat moet de nieuwe regering", max_length=30, num_return_sequences=5)

[{'generated_text': 'Dat moet de nieuwe regering-Biden dinsdag weten. �Ik begrijp dat we er nog een paar maanden mee moeten maken�, zei hij'},
 {'generated_text': 'Dat moet de nieuwe regering-Biden wel lukken om het land weer op orde te maken. �Ik kan er nog niet bij neerleggen�'},
 {'generated_text': 'Dat moet de nieuwe regering van premier Rutte, die eerder dit jaar als minister van Economische Zaken onder druk kwam te staan uit onvrede over het coronamaat'},
 {'generated_text': 'Dat moet de nieuwe regering-Biden hebben gezegd tijdens zijn persconferentie in Washington. �Maar ik vrees dat we er niet over zullen praten�'},
 {'generated_text': 'Dat moet de nieuwe regering van premier Rutte doen, stelt minister Hugo de Jonge (Volksgezondheid) in een woensdag verschenen rapport aan het Tweede Kamer. De'}]

In [25]:
Volkskrant("Volgens de demissionair premier", max_length=30, num_return_sequences=5)

[{'generated_text': 'Volgens de demissionair premier is het aantal coronapandemie-patiënten in Nederland met zo�n 60 procent gestegen sinds vorige week.'},
 {'generated_text': 'Volgens de demissionair premier is het aantal coronavaccinatiegevallen fors gedaald. De vaccinatiegraad steeg met 2 procent ten opzichte van dezelfde periode vorig'},
 {'generated_text': 'Volgens de demissionair premier Rutte en minister van Buitenlandse Zaken Ank Bijleveld (Buitenlandse zaken) wordt het �heel druk� om'},
 {'generated_text': 'Volgens de demissionair premier is er sprake van een �destructieve lockdown�, aldus Rutte. Maar hij wijst erop dat het'},
 {'generated_text': 'Volgens de demissionair premier is er een einde aan het kabinet, maar dat gaat nog altijd door. �De Kamer heeft zich vergist�'}]

In [26]:
Volkskrant("Als de formatie te lang duurt,", max_length=30, num_return_sequences=5)

[{'generated_text': 'Als de formatie te lang duurt, moet een nieuwe leider in het kabinet gaan staan. Dat is volgens VVD-leider Syrias Schimmelpenn'},
 {'generated_text': 'Als de formatie te lang duurt, kan het er nog niet worden. De PvdA-lijsttrekker Diederik Samsom en zijn coalitiepartner Jan Peter Balkenende weten dat'},
 {'generated_text': 'Als de formatie te lang duurt, moet er een nieuwe regering worden gevormd. En dat zou volgens het CDA – in tegenstelling tot vorige kabinetten �'},
 {'generated_text': 'Als de formatie te lang duurt, is er nu wel iets misgegaan. De partij van D66-voorman Sigrid Kaag (Binnenlandse Zaken)'},
 {'generated_text': 'Als de formatie te lang duurt, zal er nog maar één kandidaat worden gekozen die een duidelijke positie in het bestuurscultuur wil versterken. De oud-'}]

## Trouw

In [31]:
Trouw("Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie", max_length=30, num_return_sequences=5)

[{'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie nog niet met een nieuw kabinet. “We'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie weer op het juiste spoor. “Dit is'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie nog in lockdowns terecht. “We'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie op zoek naar een nieuwe leider. In plaats daarvan is'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie nog een tijd lang in lockdowns –'}]

In [32]:
Trouw("Dat zei Rutte vrijdag", max_length=30, num_return_sequences=5)

[{'generated_text': 'Dat zei Rutte vrijdag tijdens de wekelijkse persconferentie. “Ik denk dat we er een beetje van moeten maken�, hij voegde eraan toe.'},
 {'generated_text': 'Dat zei Rutte vrijdag na afloop van de ministerraad, in een interview met Radio 1. “Er is wel degelijk vooruitgang geboekt�, zegt'},
 {'generated_text': 'Dat zei Rutte vrijdag in een Kamerbrief aan minister Arie Slob. “Ik heb er wel begrip voor, maar het is ook belangrijk dat we'},
 {'generated_text': 'Dat zei Rutte vrijdag tijdens de persconferentie in het Catshuis. “Ik wil zeggen dat ik denk dat wij er nog heel wat van hebben�'},
 {'generated_text': 'Dat zei Rutte vrijdag in een tv-toespraak. “Ik weet het al heel goed, maar dat kan niet anders.� Hij'}]

In [33]:
Trouw("Het is een spannende race, zet Rutte,", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het is een spannende race, zet Rutte, D66 en CDA op zich af om de formatie in te voeren. “Ik ben blij dat'},
 {'generated_text': 'Het is een spannende race, zet Rutte, die in zijn eerste regeerperiode al zeven jaar aan het werk was als minister van Economische Zaken. Op de'},
 {'generated_text': 'Het is een spannende race, zet Rutte, zijn ministers en premier Mark Rutte voor elkaar in om het kabinet te laten zien dat er wel iets aan de'},
 {'generated_text': 'Het is een spannende race, zet Rutte, die in zijn derde termijn ook als premier de langstzittende VVD-leider ooit op rij staat. De'},
 {'generated_text': 'Het is een spannende race, zet Rutte, D66 en GroenLinks er samen in de peilingen. Die partijen gaan naar het noorden van Europa om zich heen'}]

In [34]:
Trouw("Tegen de achtergrond van de ernstige gevolgen van de coronacrisis", max_length=30, num_return_sequences=5)

[{'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis is een nieuw onderzoek door het Centrum voor Strategische Engineering (CSA) dat in opdracht'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn er ook nog veel mensen te weinig om zich heen, laat het RIVM weten. “'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis is nog niet duidelijk. “Voor zover bekend zijn er meer dan honderd coronatijd�'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis heeft het ministerie van VWS maandag bekendgemaakt dat er in totaal ruim 200 miljoen euro beschikbaar moet worden voor'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis hebben we nog weinig tijd. De afgelopen tien jaar is dat aantal gestegen door het coronapandem'}]

In [35]:
Trouw("Na de zomer kunnen landen uitbetaald worden", max_length=30, num_return_sequences=5)

[{'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan migranten die vanuit Griekenland naar Nederland moeten komen. Maar dat is niet voldoende, zegt een woordvoerder van het Griekse ministerie'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan mensen die in het buitenland wonen. Het ministerie van buitenlandse zaken wil dat �s werelds grootste donorlanden'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden in bitcoins of andere valuta. Maar nu zijn er al meer regels nodig, stelt een woordvoerder van ABN Amro'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan mensen die in Nederland verblijven. “We krijgen nu al veel aanvragen, maar ik heb er geen idee'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden in het Internationaal Monetair Fonds. Dat is een bedrag van bijna 1 miljard dollar (2,1 miljard euro)'}]

In [36]:
Trouw("Het demissionair kabinet heeft volgens Rutte", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het demissionair kabinet heeft volgens Rutte al een aantal maatregelen aangekondigd, waaronder meer coronapatiënten te maken met corona-risico�s.'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte niet genoeg tijd nodig. De eerste stappen zijn te volgen, zo blijkt uit de cijfers van het Centraal Bureau voor'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte en Kamervoorzitter Aung San Suu Kyi gevraagd de komende drie jaar het geldkraan uit te draaien om'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte een aantal maatregelen te nemen om het klimaatbeleid in stand te brengen, maar hij is niet optimistisch. �'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte �een heel moeilijke periode doorgemaakt�, omdat er nog niet een einde was aan de lockdown.'}]

In [37]:
Trouw("Dat moet de nieuwe regering", max_length=30, num_return_sequences=5)

[{'generated_text': 'Dat moet de nieuwe regering van president-Bush worden, maar hij heeft er geen idee hoe dat precies is. Het ministerie van buitenlandse zaken liet weten'},
 {'generated_text': 'Dat moet de nieuwe regering wel lukken. In het eerste kwartaal van 2021 is er sprake van 3,4 miljard euro aan extra uitgaven voor onderwijs en gezondheidszorg'},
 {'generated_text': 'Dat moet de nieuwe regering van premier Mark Rutte, die in zijn eerste regeringsverklaring al aangaf dat het “een goed idee� was om'},
 {'generated_text': 'Dat moet de nieuwe regering van premier Rutte weten. “Ik weet het nog niet, maar ik denk dat we wel kunnen helpen�,'},
 {'generated_text': 'Dat moet de nieuwe regering van president Bashar al-Assad doen, maar in het uiterste geval is er niet veel veranderd. “De'}]

In [38]:
Trouw("Volgens de demissionair premier", max_length=30, num_return_sequences=5)

[{'generated_text': 'Volgens de demissionair premier Mark Rutte heeft “een grote stap voorwaarts gezet� gemaakt, zo liet hij in een persverklaring weten.'},
 {'generated_text': 'Volgens de demissionair premier is het niet nodig om te beginnen met een kabinetscrisis, zoals eerder deze week. “De tijd lijkt rijp'},
 {'generated_text': 'Volgens de demissionair premier Mark Rutte is dat �onacceptabel�. De coronamaatregelen, die door het kabinet tot nu toe'},
 {'generated_text': 'Volgens de demissionair premier Mark Rutte gaat het een stuk anders dan eerder gezegd. “Ik kan me niet voorstellen dat er op dit moment'},
 {'generated_text': 'Volgens de demissionair premier wordt er een nieuwe coalitie gevormd, maar dat is in ieder geval niet zo eenvoudig als gedacht. “We hebben'}]

In [39]:
Trouw("Als de formatie te lang duurt,", max_length=30, num_return_sequences=5)

[{'generated_text': 'Als de formatie te lang duurt, is dat nog een stuk lastiger. “We hebben heel weinig vertrouwen in onze leiders�, zei dem'},
 {'generated_text': 'Als de formatie te lang duurt, moet er een einde komen aan het kabinetsperiode waarin nog steeds grote onzekerheid is ontstaan over wat daar precies mee gaat gebeuren'},
 {'generated_text': 'Als de formatie te lang duurt, komt er nog eens een nieuw kabinet-Rutte II in. Het gaat om twee kabinetten: die van Rutte III'},
 {'generated_text': 'Als de formatie te lang duurt, is het ook niet ondenkbaar dat er een nieuw kabinet van VVD�ers wordt gevormd. Rutte en D66 willen'},
 {'generated_text': 'Als de formatie te lang duurt, komt er bij het vertrek van CDA-lijsttrekker Brinkman ook nog eens iets anders. “Ik voel me'}]

## Het Parool

In [42]:
HetParool("Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie", max_length=30, num_return_sequences=5)

[{'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie weer in het nieuws. Hij zei dat “'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie met een plan. “Het is belangrijk dat'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie een beetje op vooruit, zegt hij. “'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie weer terug met een tweede kabinet. “We'},
 {'generated_text': 'Volgens demissionair premier Mark Rutte (VVD) komt Nederland na de coronapandemie een nieuw kabinet op gang. Hij zei donderdag dat hij'}]

In [43]:
HetParool("Dat zei Rutte vrijdag", max_length=30, num_return_sequences=5)

[{'generated_text': 'Dat zei Rutte vrijdag in een persconferentie aan het einde van de ministerraad: “Ik heb geen idee hoe we dat moeten aanpakken.� De'},
 {'generated_text': 'Dat zei Rutte vrijdag tijdens een Kamerdebat over het onderzoek van de Europese Commissie. “Ik vind dat we er wel aan kunnen kijken�'},
 {'generated_text': 'Dat zei Rutte vrijdag in een verklaring. “We zijn blij dat het kabinet op zoek gaat naar oplossingen om dit probleem af te lossen�'},
 {'generated_text': 'Dat zei Rutte vrijdag. “Het kabinet heeft zich al jaren bezig met het oplossen van de klimaatvraagstuk�, hij voegde eraan toe'},
 {'generated_text': 'Dat zei Rutte vrijdag in een persconferentie. “We zijn nog niet helemaal klaar met het plan van de minister, maar we weten nu zeker dat'}]

In [44]:
HetParool("Het is een spannende race, zet Rutte,", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het is een spannende race, zet Rutte, D66 en ChristenUnie op afstand. “Maar we zullen het niet doen�, roept hij'},
 {'generated_text': 'Het is een spannende race, zet Rutte, die nog altijd in de schaduw van zijn voorganger Mark Rutte staat. “Ik heb het gevoel dat'},
 {'generated_text': 'Het is een spannende race, zet Rutte, D66 en ChristenUnie de kop in. De coalitie van VVD-fractievoorzitter Sigrid Kaag (binnenlandse'},
 {'generated_text': 'Het is een spannende race, zet Rutte, D66-leider Sigrid Kaag en ChristenUnie-lijsttrekker Slob in. In de eerste helft van'},
 {'generated_text': 'Het is een spannende race, zet Rutte, de voorzitter van het Europees Parlement “met grote moeite� vast. Het gaat niet alleen om'}]

In [45]:
HetParool("Tegen de achtergrond van de ernstige gevolgen van de coronacrisis", max_length=30, num_return_sequences=5)

[{'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis is er weer een grote vraag over hoe dat kan worden voorkomen. Volgens het Centraal Bureau voor de Statistiek'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis is het nog onduidelijk wanneer er een vaccin tegen corona zal komen. Het kabinet heeft nu al aangegeven dat'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn er meer dan honderd coronadoden per jaar in Nederland, zo meldt het Centraal Bureau voor de'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis zijn er nu al twee nieuwe sterfgevallen gemeld bij jongeren, meldt het Landelijk Coördinatie Centrum (L'},
 {'generated_text': 'Tegen de achtergrond van de ernstige gevolgen van de coronacrisis komen er nog steeds nieuwe meldingen binnen. Zo is het aantal coronavaccineerden in Nederland met een'}]

In [46]:
HetParool("Na de zomer kunnen landen uitbetaald worden", max_length=30, num_return_sequences=5)

[{'generated_text': 'Na de zomer kunnen landen uitbetaald worden aan gedupeerden, aldus het Europees Hof van Justitie. “Maar dat is niet goed genoeg�,'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden. Dat gebeurde in maart 2020, toen het kabinet van toenmalig premier Mark Rutte een pakket maatregelen maakte voor coronatijd'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden, maar die moeten ze eerst in een ander land betalen. Het ministerie van financiën noemt het voorstel �volstrekt'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden in het buitenland, zo bleek uit een nieuwe studie van adviesbureau Berenschot. Het Centraal Bureau voor de Statistiek becijferde'},
 {'generated_text': 'Na de zomer kunnen landen uitbetaald worden door het Internationale Fonds voor Wederopbouw en Ontwikkeling (FWN) in een nieuw systeem, die wordt geleid door'}]

In [47]:
HetParool("Het demissionair kabinet heeft volgens Rutte", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het demissionair kabinet heeft volgens Rutte een plan om de coronamaatregelen op te krikken in 2030 met 1,5 procent ten opzichte van'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte en zijn secondanten �onmiddellijk maatregelen genomen� om de stikstofuitstoot te beperken. Dat geldt ook'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte �nationaal vertrouwenwekkende stap gezet� gemaakt om de economie aan te passen in het Klimaatakkoord van'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte �een duidelijke visie� over de aanpak van corona, zo blijkt uit het onderzoek dat donderdag verschijnt.'},
 {'generated_text': 'Het demissionair kabinet heeft volgens Rutte �onvoldoende zicht op het klimaatdoelstellingen�, zei hij donderdag tijdens een debat over de'}]

In [48]:
HetParool("Dat moet de nieuwe regering", max_length=30, num_return_sequences=5)

[{'generated_text': 'Dat moet de nieuwe regering van president Donald Trump zijn, aldus het Amerikaanse ministerie van buitenlandse handel en ontwikkeling (SAN) in een verklaring. �'},
 {'generated_text': 'Dat moet de nieuwe regering ook doen, stelt een woordvoerster van het ministerie. “We zijn nu bezig met alle mogelijke mogelijkheden om te voldoen aan'},
 {'generated_text': 'Dat moet de nieuwe regering niet wachten. Het aantal besmettingen neemt toe in het derde kwartaal van dit jaar met 3,7 procent ten opzichte van dezelfde'},
 {'generated_text': 'Dat moet de nieuwe regering, die al eerder deze week naar verluidt in gesprek was met minister Hugo de Jonge (volkshuisvesting) en staatssecretaris Wouter Kool'},
 {'generated_text': 'Dat moet de nieuwe regering van premier Mark Rutte doen. Hij heeft al beloofd dat hij als informateur in 2023 een �strategische overeenkomst�'}]

In [49]:
HetParool("Volgens de demissionair premier", max_length=30, num_return_sequences=5)

[{'generated_text': 'Volgens de demissionair premier is er nog weinig geld beschikbaar, zo�n 150 miljoen euro. De minister van financiën zei maandag te hopen dat'},
 {'generated_text': 'Volgens de demissionair premier Mark Rutte was dat niet gebeurd. Hij noemde het �een van de zwaarste dingen die ik in mijn leven heb meegemaakt'},
 {'generated_text': 'Volgens de demissionair premier, die eerder deze week in een interview zei te hebben gezegd dat hij het “oneerlijk� was om'},
 {'generated_text': 'Volgens de demissionair premier Mark Rutte kan het aantal coronapandemiepatiënten nog oplopen tot 4,550. Dat is zo�'},
 {'generated_text': 'Volgens de demissionair premier heeft Van Nieuwenhoven geen enkele reden om een nieuw kabinet te formeren. De nieuwe coalitie van VVD, D66 en'}]

In [50]:
HetParool("Als de formatie te lang duurt,", max_length=30, num_return_sequences=5)

[{'generated_text': 'Als de formatie te lang duurt, is er nog geen moment in zijn carrière getwijfeld aan het leiderschap. “Hij heeft een grote verantwoordelijkheid�'},
 {'generated_text': 'Als de formatie te lang duurt, kan het nog wel eens duren. De afgelopen twee weken zijn er tal van politieke schermutselingen tussen VVD en D66'},
 {'generated_text': 'Als de formatie te lang duurt, komt er nog geen einde aan het conflict. Het CDA-leider Syriza Sapphire zei maandag in een'},
 {'generated_text': 'Als de formatie te lang duurt, is dat niet goed voor het imago van CDA-leider Sybrand Buma�s partij. “Ik'},
 {'generated_text': 'Als de formatie te lang duurt, dan gaat het om een nieuw kabinet. Er zijn grote verschillen tussen VVD- en CDA-leider Sigrid Kaag'}]

# Sample Topic Modeling

## Vanilla

Sampling

In [ ]:
from tqdm import tqdm

In [12]:
sample_Vanilla = []
for i in tqdm(range(0, 10)):
  sample_Vanilla.extend(Vanilla("Volgens Mark Rutte", max_length=30, num_return_sequences=100))

In [13]:
print(len(sample_Vanilla))

1000


LDA Topic Modeling

In [14]:
!pip install pyLDAvis
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')
import json, os
import gensim
from gensim.utils import simple_preprocess

# regular expression library
import re
import pprint
import gensim.corpora as corpora

import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 14.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=42cfe42fd6cc0e3c7883ae26fc7d49c4ff48bbd39f790939e947833e050bc4cc
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [15]:
df_sample = pd.DataFrame(sample_Vanilla).rename(columns={'generated_text':'cleantext'})

In [18]:
def lda_analysis(frame, path_lda, topics=5):
  articles = frame[frame['cleantext'].str.len()>5]
  # Remove punctuation
  articles['cleantext_processed'] = articles['cleantext'].map(lambda x: re.sub('[,\\.!?]', '', x))
  # Save D66
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('D66', 'dzeszes', x))
  # Save BIJ1
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('BIJ1', 'bijeeen', x))
  # Save 50PLUS
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('50PLUS', 'vijfplus', x))
  # Save JA21
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('JA21', 'jatweeeen', x))
  # Save DenHaan
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('Den Haan', 'denhaan', x))
  # Convert the titles to lowercase
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: x.lower())

  p_and_p =  ['vvd', 'dzeszes', 'pvv', 'cda', 'pvda', 'sp', 'gl', 'fvd', 'pvdd',
             'cu', 'sgp', 'volt', 'jatweeeen', 'bbb', 'bijeeen', 'vijfplus', 'rutte', 'kaag',
             'wilders', 'hoekstra', 'ploumen', 'marijnissen', 'klaver', 'baudet',
             'ouwehand', 'segers', 'staaij', 'dassen', 'eerdmans', 'plas',
             'simons', 'denhaan']

  
  pat = '|'.join(r"\b{}\b".format(x) for x in p_and_p)
  articles = articles[articles['cleantext_processed'].str.contains(pat)]
  print(articles.shape)

  stop_words.extend(['nbsp', 'rsquo', 'lsquo','euml', 'rdquo', 'quot', 'ldquo',
                     'we', 'wel', 'jaar', 'zegt', 'gaat','moeten', 'volgens',
                     'heel', 'maken', 'weer', 'gaan', 'twee', 'mensen', 'waar', 'alleen',
                     'komt', 'zoals', 'alle', 'wij', 'zei','komen','tussen', 'eerste','zien'
                     'eigen', 'leven','grote', 'nieuwe','mee','goed', 'aantal','laten'
                     ,'tweede', 'af', 'eerder', 'eigen', 'elkaar','afgelopen', 'week',
                     'zeggen', 'echt', 'dag', 'zien', 'steeds', 'laten', 'willen', 'waarin', 'volgens'])

  def sent_to_words(sentences):
      for sentence in sentences:
          # deacc=True removes punctuations
          yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

  def remove_stopwords(texts):
      return [[word for word in simple_preprocess(str(doc)) 
              if word not in stop_words] for doc in texts]

  data = articles.cleantext_processed.values.tolist()
  data_words = list(sent_to_words(data))# remove stop words
  data_words = remove_stopwords(data_words)

  # Create Dictionary
  id2word = corpora.Dictionary(data_words)

  # Create Corpus
  texts = data_words

  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]

  # View
  #print(corpus[:1][0][:30])

  # number of topics
  num_topics = topics
  # Build LDA model
  lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                        id2word=id2word,
                                        num_topics=num_topics)
  pprint.pprint(lda_model.show_topics(num_words=20, formatted=False))
  doc_lda = lda_model[corpus]

  # def terms_topics_frame(terms):

  #   df = pd.DataFrame(data=None)

  #   for t in terms:
  #     df_t_topics = pd.DataFrame(lda_model.get_term_topics(t, minimum_probability=0.000000001))
  #     df_t_topics = df_t_topics.drop(columns=0)
  #     df_t_topics = df_t_topics.rename(columns={1:t})
  #     df = pd.concat([df, df_t_topics], axis=1)
    
  #   return df

  # df_tt = terms_topics_frame(['vvd','dzeszes','pvv','cda', 'pvda',  'sp','groenlinks','fvd','pvdd','cu',
  #                               'sgp','volt','jatweeeen','bbb', 'bijeeen', 'vijfplus']
  # )
  # df_tt['topic'] = df_tt.index

  # sns.set_theme(style="whitegrid")

  # # Plot the topic dist per term
  # fig = df_tt.set_index('topic').plot(kind='bar', figsize=(20,20), stacked=True)
  # fig.figure.savefig(path_plot)

  # Visualize the topics
  pyLDAvis.enable_notebook()
  LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
  LDAvis = gensimvis.prepare(lda_model, corpus, id2word, sort_topics=False)
  pyLDAvis.save_html(LDAvis, path_lda)

In [21]:
lda_analysis(df_sample,'drive/MyDrive/CS/Thesis/sample_Vanilla.html',topics=5)

(1000, 2)


/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

[(0,
  [('rutte', 0.11009157),
   ('mark', 0.106636055),
   ('vvd', 0.02727622),
   ('zaken', 0.016673507),
   ('minister', 0.01612214),
   ('kabinet', 0.01358605),
   ('nederland', 0.013456484),
   ('weet', 0.011916588),
   ('vind', 0.011509195),
   ('buitenlandse', 0.011199128),
   ('premier', 0.010856993),
   ('blij', 0.00880259),
   ('denk', 0.007605461),
   ('jammer', 0.0066289315),
   ('weten', 0.0064939396),
   ('tijd', 0.006362742),
   ('belangrijk', 0.006131967),
   ('interview', 0.0051102727),
   ('debat', 0.004972607),
   ('pvda', 0.004880398)]),
 (1,
  [('rutte', 0.07519039),
   ('mark', 0.07312332),
   ('zaken', 0.023091892),
   ('minister', 0.01959683),
   ('voorzitter', 0.01881168),
   ('kabinet', 0.016177522),
   ('vvd', 0.014936921),
   ('economische', 0.012366077),
   ('nederland', 0.011076658),
   ('premier', 0.01089048),
   ('parlement', 0.008689319),
   ('buitenlandse', 0.008084064),
   ('blij', 0.007829242),
   ('goede', 0.0074479883),
   ('moeilijke', 0.007139870

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


## General

In [ ]:
sample_General = []
for i in tqdm(range(0, 10)):
  sample_General.extend(General("Volgens Mark Rutte", max_length=30, num_return_sequences=100))

In [ ]:
df_sample = pd.DataFrame(sample_General).rename(columns={'generated_text':'cleantext'})

In [ ]:
lda_analysis(df_sample,'drive/MyDrive/CS/Thesis/sample_General.html',topics=10)

## NRC

In [ ]:
sample_NRC = []
for i in tqdm(range(0, 10)):
  sample_NRC.extend(NRC("Volgens Mark Rutte", max_length=30, num_return_sequences=100))

In [ ]:
df_sample = pd.DataFrame(sample_NRC).rename(columns={'generated_text':'cleantext'})

In [ ]:
lda_analysis(df_sample,'drive/MyDrive/CS/Thesis/sample_NRC.html',topics=10)

## Volkskrant

In [ ]:
sample_Volkskrant = []
for i in tqdm(range(0, 10)):
  sample_Volkskrant.extend(Volkskrant("Volgens Mark Rutte", max_length=30, num_return_sequences=100))

In [ ]:
df_sample = pd.DataFrame(sample_NRC).rename(columns={'generated_text':'cleantext'})

In [ ]:
lda_analysis(df_sample,'drive/MyDrive/CS/Thesis/sample_NRC.html',topics=10)